## label_csv formatting 

In [2]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/research'

In [3]:
import os
os.chdir("../")


In [4]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset'

In [7]:
import pandas as pd
from pathlib import Path

def clean_csv(csv_path: Path):
    df = pd.read_csv(csv_path, header=None, names=['image', 'label'])
    df['image'] = df['image'].apply(lambda x: Path(x).name)
    df.to_csv(csv_path, index=False)

clean_csv(Path("artifacts/activity_labels_train.csv"))
clean_csv(Path("artifacts/activity_labels_test.csv"))
# If needed: clean_csv(Path("activity_labels_val.csv"))


# TRADITIONAL mlp and random forest

In [ ]:
"""
Thermal Pose HAR Pipeline
Author: priyam
Description:
    - Extracts 51‑dim keypoint feature vectors from thermal images using a trained YOLOv8‑Pose model.
    - Trains a classification model (MLP or Random Forest) on those vectors to predict activity labels.
    - Evaluates the classifier on the held‑out test split.
    - Saves the evaluation report to artifacts/classifiers/eval_reports/ with filename format: MODELNAME_evaluation.txt

Dataset structure (expected):
    project_root/
        artifacts/OPEN_THERMAL_IMAGE/
            train/images/   (thermal .png/.jpg files)
            test/images/
        activity_labels_train.csv   (cols: image,label)
        activity_labels_test.csv
        weights/yolov8‑pose.pt      (trained pose model)

All paths are configurable via the Config dataclass.
"""

from dataclasses import dataclass, field
from pathlib import Path
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics import YOLO
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib


# ----------------------------------------------------------------------
#  Configuration
# ----------------------------------------------------------------------
@dataclass
class Config:
    project_root: Path = Path('.').resolve()

    # Pose model
    yolo_weights: Path = Path('artifacts/weights/best.pt')

    # Image folders
    train_dir: Path = Path('artifacts/OPEN_THERMAL_IMAGE/train/images')
    test_dir: Path = Path('artifacts/OPEN_THERMAL_IMAGE/test/images')

    # CSVs: <image file name>,<label>
    train_csv: Path = Path('artifacts/activity_labels_train.csv')
    test_csv: Path = Path('artifacts/activity_labels_test.csv')

    # Output
    feature_dir: Path = Path('artifacts/pose_features')
    model_dir: Path = Path('artifacts/classifiers')
    report_dir: Path = Path('artifacts/classifiers/eval_reports')

    img_size: int = 640                     # inference size for YOLO
    device: str = 'cpu'                     # or 'cuda'

    def resolve_paths(self):
        # Make all paths absolute relative to project_root
        for attrib in ('yolo_weights', 'train_dir', 'test_dir',
                       'train_csv', 'test_csv',
                       'feature_dir', 'model_dir', 'report_dir'):
            path = getattr(self, attrib)
            if not path.is_absolute():
                setattr(self, attrib, (self.project_root / path).resolve())

        self.feature_dir.mkdir(parents=True, exist_ok=True)
        self.model_dir.mkdir(parents=True, exist_ok=True)
        self.report_dir.mkdir(parents=True, exist_ok=True)


# ----------------------------------------------------------------------
#  Pose extraction
# ----------------------------------------------------------------------
class PoseExtractor:
    """Runs YOLOv8‑Pose on images and returns a 51‑dim keypoint vector."""
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.model = YOLO(str(cfg.yolo_weights))
        self.model.fuse()  # speed

    def _image_to_vec(self, img_path: Path) -> np.ndarray | None:
        # Inference
        results = self.model.predict(str(img_path),
                                     imgsz=self.cfg.img_size,
                                     device=self.cfg.device,
                                     verbose=False)
        if not results or len(results[0].keypoints) == 0:
            return None

        # Only the first detected person (index 0).
        kp_xy = results[0].keypoints.xy[0].cpu().numpy()      # (17,2)
        kp_conf = results[0].keypoints.conf[0].cpu().numpy()  # (17,)

        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        kp_xy_norm = kp_xy / np.array([[w, h]])

        vec = np.concatenate([kp_xy_norm.flatten(), kp_conf])   # 17*2 + 17 = 51
        return vec.astype(np.float32)

    def extract_split(self, split: str, force: bool = False):
        """Returns (X, y) for a given split, caching .npy/.csv."""
        img_dir = getattr(self.cfg, f'{split}_dir')
        csv_file = getattr(self.cfg, f'{split}_csv')
        feat_npy = self.cfg.feature_dir / f'{split}.npy'
        lbl_csv = self.cfg.feature_dir / f'{split}_labels.csv'

        if feat_npy.exists() and lbl_csv.exists() and not force:
            return np.load(feat_npy), pd.read_csv(lbl_csv)['label'].to_numpy()

        df = pd.read_csv(csv_file)
        vectors, labels = [], []

        for _, row in tqdm(df.iterrows(), total=len(df), desc=f'Extracting {split}'):
            img_fp = img_dir / row['image']
            if not img_fp.exists():
                print(f'Skip missing {img_fp}')
                continue
            vec = self._image_to_vec(img_fp)
            if vec is not None:
                vectors.append(vec)
                labels.append(row['label'])

        X = np.vstack(vectors)
        y = np.array(labels)

        np.save(feat_npy, X)
        pd.DataFrame({'label': y}).to_csv(lbl_csv, index=False)
        return X, y


# ----------------------------------------------------------------------
#  Classifier factory
# ----------------------------------------------------------------------
class ClassifierFactory:
    @staticmethod
    def get(name: str):
        name = name.lower()
        if name == 'mlp':
            return MLPClassifier(hidden_layer_sizes=(128, 64),
                                 activation='relu',
                                 solver='adam',
                                 max_iter=500,
                                 random_state=42)
        if name in ('rf', 'randomforest', 'random_forest'):
            return RandomForestClassifier(n_estimators=400,
                                          max_depth=None,
                                          n_jobs=-1,
                                          random_state=42)
        raise ValueError(f'Unknown classifier {name}')


# ----------------------------------------------------------------------
#  Training / evaluation pipeline
# ----------------------------------------------------------------------
class HARPipeline:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.cfg.resolve_paths()
        self.extractor = PoseExtractor(cfg)

    def run(self, clf_name: str = 'mlp', force_extract: bool = False):
        # 1. Feature extraction
        X_train, y_train = self.extractor.extract_split('train', force_extract)
        X_test,  y_test  = self.extractor.extract_split('test',  force_extract)

        # 2. Train classifier
        clf = ClassifierFactory.get(clf_name)
        clf.fit(X_train, y_train)

        # 3. Save
        model_path = self.cfg.model_dir / f'{clf_name}.pkl'
        joblib.dump(clf, model_path)
        print(f'[✓] Saved trained classifier -> {model_path}')

        # 4. Evaluate
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        print(f'Test accuracy: {acc:.4f}')
        print(report)

        # Save report
        report_path = self.cfg.report_dir / f'{clf_name}_evaluation.txt'
        with open(report_path, 'w') as f:
            f.write(f'Accuracy: {acc:.4f}\n')
            f.write(report)
        print(f'[✓] Saved evaluation report -> {report_path}')

        return clf


# ----------------------------------------------------------------------
#  Script entry‑point
# ----------------------------------------------------------------------
def main(clf='mlp', force=False, device='cpu', yolo_weights=None):
    cfg = Config()
    if yolo_weights:
        cfg.yolo_weights = Path(yolo_weights)
    cfg.device = device

    pipeline = HARPipeline(cfg)
    pipeline.run(clf, force)

# Use this inside notebook directly:
main(clf='mlp', force=True)  # or force=False


YOLOv8n-pose summary (fused): 81 layers, 3,289,964 parameters, 0 gradients, 9.2 GFLOPs


Extracting test: 100%|██████████| 2151/2151 [03:31<00:00, 10.15it/s] 


[✓] Saved trained classifier -> /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/classifiers/mlp.pkl
Test accuracy: 0.8145
                     precision    recall  f1-score   support

EXERCISE_BODY_SWING       0.78      0.86      0.82       185
   LOOKING_STRAIGHT       0.90      0.99      0.95      1081
   SITTING_STANDING       0.17      0.47      0.25        15
           STANDING       0.00      0.00      0.00       129
           fighting       0.57      0.22      0.32        36
          gesturing       0.22      0.76      0.34        34
            walking       0.82      0.71      0.76       671

           accuracy                           0.81      2151
          macro avg       0.50      0.57      0.49      2151
       weighted avg       0.79      0.81      0.80      2151

[✓] Saved evaluation report -> /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/classifiers/eval_reports/mlp_evaluation.txt


# random forest

In [6]:
"""
Thermal Pose HAR Pipeline
Author: priyam
Description:
    - Extracts 51‑dim keypoint feature vectors from thermal images using a trained YOLOv8‑Pose model.
    - Trains a classification model (MLP or Random Forest) on those vectors to predict activity labels.
    - Evaluates the classifier on the held‑out test split.
    - Saves the evaluation report to artifacts/classifiers/eval_reports/ with filename format: MODELNAME_evaluation.txt

Dataset structure (expected):
    project_root/
        artifacts/OPEN_THERMAL_IMAGE/
            train/images/   (thermal .png/.jpg files)
            test/images/
        activity_labels_train.csv   (cols: image,label)
        activity_labels_test.csv
        weights/yolov8‑pose.pt      (trained pose model)

All paths are configurable via the Config dataclass.
"""

from dataclasses import dataclass, field
from pathlib import Path
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics import YOLO
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib


# ----------------------------------------------------------------------
#  Configuration
# ----------------------------------------------------------------------
@dataclass
class Config:
    project_root: Path = Path('.').resolve()

    # Pose model
    yolo_weights: Path = Path('artifacts/weights/best.pt')

    # Image folders
    train_dir: Path = Path('artifacts/OPEN_THERMAL_IMAGE/train/images')
    test_dir: Path = Path('artifacts/OPEN_THERMAL_IMAGE/test/images')

    # CSVs: <image file name>,<label>
    train_csv: Path = Path('artifacts/activity_labels_train.csv')
    test_csv: Path = Path('artifacts/activity_labels_test.csv')

    # Output
    feature_dir: Path = Path('artifacts/pose_features')
    model_dir: Path = Path('artifacts/classifiers')
    report_dir: Path = Path('artifacts/classifiers/eval_reports')

    img_size: int = 640                     # inference size for YOLO
    device: str = 'cpu'                     # or 'cuda'

    def resolve_paths(self):
        # Make all paths absolute relative to project_root
        for attrib in ('yolo_weights', 'train_dir', 'test_dir',
                       'train_csv', 'test_csv',
                       'feature_dir', 'model_dir', 'report_dir'):
            path = getattr(self, attrib)
            if not path.is_absolute():
                setattr(self, attrib, (self.project_root / path).resolve())

        self.feature_dir.mkdir(parents=True, exist_ok=True)
        self.model_dir.mkdir(parents=True, exist_ok=True)
        self.report_dir.mkdir(parents=True, exist_ok=True)


# ----------------------------------------------------------------------
#  Pose extraction
# ----------------------------------------------------------------------
class PoseExtractor:
    """Runs YOLOv8‑Pose on images and returns a 51‑dim keypoint vector."""
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.model = YOLO(str(cfg.yolo_weights))
        self.model.fuse()  # speed

    def _image_to_vec(self, img_path: Path) -> np.ndarray | None:
        # Inference
        results = self.model.predict(str(img_path),
                                     imgsz=self.cfg.img_size,
                                     device=self.cfg.device,
                                     verbose=False)
        if not results or len(results[0].keypoints) == 0:
            return None

        # Only the first detected person (index 0).
        kp_xy = results[0].keypoints.xy[0].cpu().numpy()      # (17,2)
        kp_conf = results[0].keypoints.conf[0].cpu().numpy()  # (17,)

        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        kp_xy_norm = kp_xy / np.array([[w, h]])

        vec = np.concatenate([kp_xy_norm.flatten(), kp_conf])   # 17*2 + 17 = 51
        return vec.astype(np.float32)

    def extract_split(self, split: str, force: bool = False):
        """Returns (X, y) for a given split, caching .npy/.csv."""
        img_dir = getattr(self.cfg, f'{split}_dir')
        csv_file = getattr(self.cfg, f'{split}_csv')
        feat_npy = self.cfg.feature_dir / f'{split}.npy'
        lbl_csv = self.cfg.feature_dir / f'{split}_labels.csv'

        if feat_npy.exists() and lbl_csv.exists() and not force:
            return np.load(feat_npy), pd.read_csv(lbl_csv)['label'].to_numpy()

        df = pd.read_csv(csv_file)
        vectors, labels = [], []

        for _, row in tqdm(df.iterrows(), total=len(df), desc=f'Extracting {split}'):
            img_fp = img_dir / row['image']
            if not img_fp.exists():
                print(f'Skip missing {img_fp}')
                continue
            vec = self._image_to_vec(img_fp)
            if vec is not None:
                vectors.append(vec)
                labels.append(row['label'])

        X = np.vstack(vectors)
        y = np.array(labels)

        np.save(feat_npy, X)
        pd.DataFrame({'label': y}).to_csv(lbl_csv, index=False)
        return X, y


# ----------------------------------------------------------------------
#  Classifier factory
# ----------------------------------------------------------------------
class ClassifierFactory:
    @staticmethod
    def get(name: str):
        name = name.lower()
        if name == 'mlp':
            return MLPClassifier(hidden_layer_sizes=(128, 64),
                                 activation='relu',
                                 solver='adam',
                                 max_iter=500,
                                 random_state=42)
        if name in ('rf', 'randomforest', 'random_forest'):
            return RandomForestClassifier(n_estimators=400,
                                          max_depth=None,
                                          n_jobs=-1,
                                          random_state=42)
        raise ValueError(f'Unknown classifier {name}')


# ----------------------------------------------------------------------
#  Training / evaluation pipeline
# ----------------------------------------------------------------------
class HARPipeline:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.cfg.resolve_paths()
        self.extractor = PoseExtractor(cfg)

    def run(self, clf_name: str = 'mlp', force_extract: bool = False):
        # 1. Feature extraction
        X_train, y_train = self.extractor.extract_split('train', force_extract)
        X_test,  y_test  = self.extractor.extract_split('test',  force_extract)

        # 2. Train classifier
        clf = ClassifierFactory.get(clf_name)
        clf.fit(X_train, y_train)

        # 3. Save
        model_path = self.cfg.model_dir / f'{clf_name}.pkl'
        joblib.dump(clf, model_path)
        print(f'[✓] Saved trained classifier -> {model_path}')

        # 4. Evaluate
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        print(f'Test accuracy: {acc:.4f}')
        print(report)

        # Save report
        report_path = self.cfg.report_dir / f'{clf_name}_evaluation.txt'
        with open(report_path, 'w') as f:
            f.write(f'Accuracy: {acc:.4f}\n')
            f.write(report)
        print(f'[✓] Saved evaluation report -> {report_path}')

        return clf


# ----------------------------------------------------------------------
#  Script entry‑point
# ----------------------------------------------------------------------
def main(clf='rf', force=True, device='cpu', yolo_weights=None):
    cfg = Config()
    if yolo_weights:
        cfg.yolo_weights = Path(yolo_weights)
    cfg.device = device

    pipeline = HARPipeline(cfg)
    pipeline.run(clf, force)

# Use this inside notebook directly:
main(clf='mlp', force=True)  # or force=False


YOLOv8n-pose summary (fused): 81 layers, 3,289,964 parameters, 0 gradients, 9.2 GFLOPs


Extracting test: 100%|██████████| 2151/2151 [02:50<00:00, 12.60it/s]


[✓] Saved trained classifier -> /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/classifiers/mlp.pkl
Test accuracy: 0.8145
                     precision    recall  f1-score   support

EXERCISE_BODY_SWING       0.78      0.86      0.82       185
   LOOKING_STRAIGHT       0.90      0.99      0.95      1081
   SITTING_STANDING       0.17      0.47      0.25        15
           STANDING       0.00      0.00      0.00       129
           fighting       0.57      0.22      0.32        36
          gesturing       0.22      0.76      0.34        34
            walking       0.82      0.71      0.76       671

           accuracy                           0.81      2151
          macro avg       0.50      0.57      0.49      2151
       weighted avg       0.79      0.81      0.80      2151

[✓] Saved evaluation report -> /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/classifiers/eval_reports/mlp_evaluation.txt
